<a href="https://colab.research.google.com/github/Debayan2004/BR_Segmentation/blob/main/Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import os
import nibabel as nib
import numpy as np
import random
from scipy.ndimage import rotate, zoom
from skimage import exposure
from skimage.transform import resize  # Import resize for rescaling

# Define paths
main_dir = '/content/drive/MyDrive/BS_2018/training_dataset'
new_folder = os.path.join(main_dir, '107')  # New folder to store augmented data

# Create new folder structure
if not os.path.exists(new_folder):
    os.makedirs(new_folder)

for sub_folder in ['pre', 'segm.nii']:
    os.makedirs(os.path.join(new_folder, sub_folder), exist_ok=True)

# Target size for resizing
target_size = (240, 240, 48)

# Augmentation functions
def adjust_intensity(image):
    """ Adjust intensity using histogram equalization or contrast adjustment. """
    return exposure.equalize_hist(image)

def rotate_image(image, angle):
    """ Rotate the image randomly by a certain angle. """
    return rotate(image, angle, axes=(0, 1), reshape=False, order=1)

def flip_image(image):
    """ Flip the image randomly along x, y, or z axis. """
    flip_axis = random.choice([0, 1, 2])
    return np.flip(image, axis=flip_axis)

def zoom_image(image, zoom_factor=1.1):
    """ Apply zoom to the image. """
    return zoom(image, zoom_factor)

def augment_image(image):
    """ Apply a combination of augmentation techniques. """
    image = adjust_intensity(image)
    if random.random() > 0.5:
        image = rotate_image(image, angle=random.uniform(-20, 20))
    if random.random() > 0.5:
        image = flip_image(image)
    if random.random() > 0.5:
        image = zoom_image(image, zoom_factor=random.uniform(0.9, 1.1))
    return image

def resize_image(image):
    """ Resize the image to the target size. """
    return resize(image, target_size, mode='constant', preserve_range=True)

# Process folders
folders = ['1', '14', '27', '29', '4', '5', '7']

for folder in folders:
    pre_dir = os.path.join(main_dir, folder, 'pre')
    seg_dir = os.path.join(main_dir, folder, 'segm.nii')

    # Augment pre MRI images (FLAIR, T1, IR, reg_T1, reg_IR)
    for image_name in ['FLAIR.nii/FLAIR.nii', 'T1.nii/T1.nii', 'IR.nii/IR.nii', 'reg_T1.nii/reg_T1.nii', 'reg_IR.nii/reg_IR.nii']:
        image_path = os.path.join(pre_dir, image_name)
        image_data = nib.load(image_path).get_fdata()

        # Apply augmentation
        augmented_image = augment_image(image_data)
        # Resize augmented image
        resized_image = resize_image(augmented_image)

        # Create subfolder structure in the new directory
        sub_folder_path = os.path.join(new_folder, 'pre', os.path.dirname(image_name))
        if not os.path.exists(sub_folder_path):
            os.makedirs(sub_folder_path)

        # Save augmented image to new folder
        new_image_path = os.path.join(sub_folder_path, os.path.basename(image_name))  # Use the full path with folder
        nib.save(nib.Nifti1Image(resized_image, None), new_image_path)

    # Augment segmentation mask
    seg_path = os.path.join(seg_dir, 'segm.nii')
    seg_data = nib.load(seg_path).get_fdata()

    # Augment segmentation mask (use same transformations as for images)
    augmented_seg = augment_image(seg_data)
    # Resize augmented segmentation mask
    resized_seg = resize_image(augmented_seg)

    # Save augmented segmentation mask
    new_seg_path = os.path.join(new_folder, 'segm.nii', 'segm.nii')
    nib.save(nib.Nifti1Image(resized_seg, None), new_seg_path)

print(f"Augmented data stored in {new_folder}")


Augmented data stored in /content/drive/MyDrive/BS_2018/training_dataset/107
